# Import

In [30]:
import numpy as np
import pandas as pd

# Data preporation

In [31]:
# Load previous day data
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4748,2020-05-29,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,105.0,0.0,12.0,2133.0,17.0,728.0
4749,2020-05-29,Чеченская Республика,Chechen Republic,95.0,18.0,0.0,42.0,1192.0,13.0,787.0
4750,2020-05-29,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,7.0,0.0,10.0,73.0,1.0,50.0
4751,2020-05-29,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,31.0,2.0,42.0,2280.0,9.0,898.0
4752,2020-05-29,Ярославская область,Yaroslavl region,76.0,84.0,1.0,45.0,2976.0,14.0,698.0


In [32]:
# Read update data
# Result of data_parsing.ipynb
file_name = 'covid19-russia-cases-upd.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,178196.0,2020-05-30,2408.0,78324.0,2367.0,78.0,3599.0
30,Московская область,38238.0,2020-05-30,424.0,7720.0,735.0,24.0,387.0
64,Санкт-Петербург,15580.0,2020-05-30,191.0,5373.0,365.0,13.0,260.0
33,Нижегородская область,9533.0,2020-05-30,93.0,3637.0,289.0,4.0,171.0
67,Свердловская область,5184.0,2020-05-30,26.0,2142.0,261.0,1.0,145.0


In [33]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Updating

In [34]:
def upd(row):
    reg = row['Region/City']
    
    row['Confirmed'] = rus_df[rus_df['Region/City'] == reg]['Confirmed'].max() + row['Day-Confirmed']
    row['Deaths'] = rus_df[rus_df['Region/City'] == reg]['Deaths'].max() + row['Day-Deaths']
    row['Recovered'] = rus_df[rus_df['Region/City'] == reg]['Recovered'].max() + row['Day-Recovered']
    
    row['Confirmed'] = row['Day-Confirmed'] if np.isnan(row['Confirmed']) else row['Confirmed']
    row['Deaths'] = row['Day-Deaths'] if np.isnan(row['Deaths']) else row['Deaths']
    row['Recovered'] = row['Day-Recovered'] if np.isnan(row['Recovered']) else row['Recovered']
    
    return row.drop('Region/City')

# If cumulative values unknown
# upd_df = upd_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [35]:
# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

In [36]:
# Region names translation
# from googletrans import Translator
# translator = Translator()

# def translate(rus):
#     eng = translator.translate(rus, src='ru', dest='en')
#     return eng.text

# upd_df['Region/City-Eng'] = upd_df['Region/City'].apply(translate)

In [37]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [38]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4828,2020-05-30,Тюменская область,Tyumen region,72.0,28.0,1.0,7.0,1608.0,8.0,790.0
4829,2020-05-30,Удмуртская Республика,Udmurt republic,18.0,24.0,0.0,15.0,629.0,12.0,410.0
4830,2020-05-30,Ульяновская область,Ulyanovsk region,73.0,98.0,0.0,66.0,2369.0,8.0,700.0
4831,2020-05-30,Хабаровский край,Khabarovsk region,27.0,59.0,0.0,33.0,2018.0,11.0,958.0
4832,2020-05-30,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,167.0,1.0,24.0,2300.0,18.0,752.0
4833,2020-05-30,Челябинская область,Chelyabinsk region,74.0,292.0,2.0,152.0,2878.0,17.0,1258.0
4834,2020-05-30,Чеченская Республика,Chechen Republic,95.0,17.0,0.0,38.0,1209.0,13.0,825.0
4835,2020-05-30,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,5.0,0.0,3.0,78.0,1.0,53.0
4836,2020-05-30,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,30.0,0.0,55.0,2310.0,9.0,953.0
4837,2020-05-30,Ярославская область,Yaroslavl region,76.0,79.0,0.0,53.0,3055.0,14.0,751.0


In [39]:
# Negative values fixing
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(f"Conf {diff_c} Death {diff_d} Rec {diff_r} {last['Region/City']}")
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [40]:
# Manual fixes if needed
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0


# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [41]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 396913.0 396913.0
Russia Deaths: 4558.0 4558.0
Russia Recovered: 167683.0 167683.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [42]:
file_name = 'release/covid19-russia-cases.csv'
rus_df.to_csv(file_name, index=False)